<a href="https://colab.research.google.com/github/luisscrc/ProyectoFinalIACENTER/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git ## LIBRERÍA PARA ACELERAR POR GPU
!python rapidsai-csp-utils/colab/pip-install.py ## COMPILAR CON T4

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 395 (delta 92), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (395/395), 108.50 KiB | 13.56 MiB/s, done.
Resolving deltas: 100% (194/194), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!wget https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico ## Comando para descargar algo de una pagina web

--2023-11-13 01:51:24--  https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico
Resolving datos.gob.mx (datos.gob.mx)... 96.17.96.19, 96.17.96.17
Connecting to datos.gob.mx (datos.gob.mx)|96.17.96.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21595 (21K) [text/html]
Saving to: ‘informacion-referente-a-casos-covid-19-en-mexico’

informacion-referen 100%[===================>]  21.09K  --.-KB/s    in 0s      

2023-11-13 01:51:25 (339 MB/s) - ‘informacion-referente-a-casos-covid-19-en-mexico’ saved [21595/21595]



In [9]:
!unzip /content/datos_abiertos_covid19.zip

Archive:  /content/datos_abiertos_covid19.zip
  inflating: COVID19MEXICO.csv       


In [58]:
import pandas as pd
import numpy as np
data=pd.read_csv('COVID19MEXICO.csv',low_memory=False) # RAM INFINITAAAAA
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [59]:
df=data

In [60]:
df = df.drop(columns=['SECTOR','MUNICIPIO_RES','HABLA_LENGUA_INDIG','ID_REGISTRO','ORIGEN','PAIS_NACIONALIDAD','PAIS_ORIGEN','INDIGENA','ENTIDAD_UM','ENTIDAD_NAC','NACIONALIDAD','ENTIDAD_RES'])
df.columns

  ## FECHA DE ACTUALIZACION; 4 TIPOS DE FECHAS
df['FECHA_ACTUALIZACION']=pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO']=pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS']=pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF']=pd.to_datetime(df['FECHA_DEF'],errors='coerce')

for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)

from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()


for col in ['TIPO_PACIENTE','SEXO']:
  df[col]=le.fit_transform(df[col])
## Características para darle valor a las fechas
df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION']- df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZACION']=(df['FECHA_ACTUALIZACION']- df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION']- df['FECHA_SINTOMAS']).dt.days
# Red NUERONAL PARA OTORGAR UN 'NaN' y evitar problemas con datos basura
df['DIAS_DEFUNCION'] = df['DIAS_DEFUNCION'].apply(lambda x: 0 if x < 0 else x)

df = df.drop(columns=['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF','DIAS_DEFUNCION'])

X=df.drop(columns=['CLASIFICACION_FINAL'])
y=df['CLASIFICACION_FINAL']

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [62]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 26 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   SEXO                   1186694 non-null  int64
 1   TIPO_PACIENTE          1186694 non-null  int64
 2   INTUBADO               1186694 non-null  int64
 3   NEUMONIA               1186694 non-null  int64
 4   EDAD                   1186694 non-null  int64
 5   EMBARAZO               1186694 non-null  int64
 6   DIABETES               1186694 non-null  int64
 7   EPOC                   1186694 non-null  int64
 8   ASMA                   1186694 non-null  int64
 9   INMUSUPR               1186694 non-null  int64
 10  HIPERTENSION           1186694 non-null  int64
 11  OTRA_COM               1186694 non-null  int64
 12  CARDIOVASCULAR         1186694 non-null  int64
 13  OBESIDAD               1186694 non-null  int64
 14  RENAL_CRONICA          1186694 non-null  int64
 15

In [63]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.19      0.04      0.07      1673
           2       1.00      0.20      0.33         5
           3       1.00      1.00      1.00     84200
           4       0.10      0.03      0.04        35
           5       0.53      0.52      0.52       396
           6       0.95      0.96      0.96     12816
           7       0.99      1.00      0.99    138214

    accuracy                           0.99    237339
   macro avg       0.68      0.54      0.56    237339
weighted avg       0.98      0.99      0.99    237339



In [64]:
import cuml
from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
#
X_train=X_train.astype('float32') ## Acelera para procesamiento por GPU
y_train=y_train.astype('float32')

clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test,y_pred))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.94      0.02      0.03      1673
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     84200
           4       0.00      0.00      0.00        35
           5       0.58      0.30      0.40       396
           6       0.95      0.97      0.96     12816
           7       0.99      1.00      0.99    138214

    accuracy                           0.99    237339
   macro avg       0.64      0.47      0.48    237339
weighted avg       0.99      0.99      0.99    237339



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

cuml_RFC = RandomForestClassifier(random_state=42)
cuml_KNC = KNeighborsClassifier()

classifiers = [ # lista
    {'Random Forest': cuml_RFC},
    {'KNN': cuml_KNC}
]

for classifier in classifiers:
    name, clf = list(classifier.items())[0]  # Extrae el nombre y clasificador del diccionario
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f'Modelo {name} resultados')
    print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


Modelo Random Forest resultados


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.94      0.02      0.03      1673
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     84200
           4       0.00      0.00      0.00        35
           5       0.58      0.30      0.40       396
           6       0.95      0.97      0.96     12816
           7       0.99      1.00      0.99    138214

    accuracy                           0.99    237339
   macro avg       0.64      0.47      0.48    237339
weighted avg       0.99      0.99      0.99    237339

Modelo KNN resultados


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.34      0.02      0.03      1673
           2       0.00      0.00      0.00         5
           3       0.95      0.84      0.89     84200
           4       0.00      0.00      0.00        35
           5       0.19      0.02      0.03       396
           6       0.96      0.92      0.94     12816
           7       0.90      0.97      0.93    138214

    accuracy                           0.92    237339
   macro avg       0.48      0.40      0.40    237339
weighted avg       0.91      0.92      0.91    237339



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [72]:
import locale # configurar compatibilidad de gradio
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [73]:
!pip install gradio # para hacer las paginas web

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 26 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   SEXO                   1186694 non-null  int64
 1   TIPO_PACIENTE          1186694 non-null  int64
 2   INTUBADO               1186694 non-null  int64
 3   NEUMONIA               1186694 non-null  int64
 4   EDAD                   1186694 non-null  int64
 5   EMBARAZO               1186694 non-null  int64
 6   DIABETES               1186694 non-null  int64
 7   EPOC                   1186694 non-null  int64
 8   ASMA                   1186694 non-null  int64
 9   INMUSUPR               1186694 non-null  int64
 10  HIPERTENSION           1186694 non-null  int64
 11  OTRA_COM               1186694 non-null  int64
 12  CARDIOVASCULAR         1186694 non-null  int64
 13  OBESIDAD               1186694 non-null  int64
 14  RENAL_CRONICA          1186694 non-null  int64
 15

In [75]:
def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo, diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno, clasificacion_final,
            migrante, uci, dias_hospitalizacion, dias_sintomas):


SyntaxError: ignored